# Predictions 
## Niccolò Simonato 
## Data & Web Mining, Academic Year 2021-2022

## Importing the dependencies and the cleaned dataset

The cleaned dataset is now imported.

The first snipped is intended to be used in the Google Drive environment, just set the path variable as needed.

The second one is intended to be used in the Jupyter Notebook environment.

In [5]:
# dependencies
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn.metrics import accuracy_score

In [6]:
# from google.colab import drive
# drive.mount('/gdrive')
# path = '/gdrive/MyDrive/Progetto DWM/Data/*.csv'
# %cd /gdrive

In [7]:
path = 'Data/'

# cleaned_df = pd.read_csv(path, low_memory = False)
train_datasets = []
test_datasets = []

for i in range(5):
    train_datasets.append(pd.read_csv(f"{path}train_dataset_2016_{i + 1}.csv", low_memory = True))
    test_datasets.append(pd.read_csv(f"{path}test_dataset_2016_{i + 1}.csv", low_memory = True))

## Data Validation
In order to prevent any exception to be raised, we will check the state of each dataset and will ensure that the data can be used.

In [8]:
train_datasets[1].describe(include='all')

,Unnamed: 0,parcelid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,N-LogTaxScore,N-zip_count,N-city_count,N-ACInd,N-HeatInd,N-PropType,N-Dev-structuretaxvaluedollarcnt,DD,MM,N-DaysInCurrentYear
count,68208.000000,6.820800e+04,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,...,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.000000,68208.00000,68208.000000,68208.000000
mean,49638.060931,1.300474e+07,3.945578,2.270320,3.003636,5.521900,2.288236,1743.108272,1743.108272,6049.177223,...,9.147980,324.725560,4834.738476,0.329390,0.654718,0.051050,0.530811,16.36267,5.867875,164.342922
std,27218.525092,2.371372e+06,2.279430,0.940313,1.009035,1.529856,1.017559,905.997275,905.997275,20.860330,...,0.681050,177.343462,7380.803303,0.469995,0.475464,0.314264,0.757977,9.00774,2.803042,85.483391
min,0.000000,1.071174e+07,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,6037.000000,...,4.824880,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,1.000000
25%,31590.000000,1.154383e+07,1.000000,2.000000,2.000000,4.000000,2.000000,1171.000000,1171.000000,6037.000000,...,8.754919,198.000000,432.000000,0.000000,0.000000,0.000000,0.198509,8.00000,4.000000,97.000000
50%,54141.500000,1.258943e+07,5.000000,2.000000,3.000000,5.565407,2.000000,1515.000000,1515.000000,6037.000000,...,9.181586,296.000000,992.000000,0.000000,1.000000,0.000000,0.405284,16.00000,6.000000,167.000000
75%,72214.250000,1.425394e+07,5.000000,3.000000,4.000000,7.000000,3.000000,2055.000000,2055.000000,6059.000000,...,9.559402,399.000000,2703.000000,1.000000,1.000000,0.000000,0.648437,24.00000,8.000000,232.000000
max,90272.000000,1.629608e+08,13.000000,20.000000,10.000000,12.000000,20.000000,20013.000000,20013.000000,6111.000000,...,12.951033,902.000000,18697.000000,1.000000,1.000000,2.000000,39.442360,31.00000,12.000000,365.000000


Since we don't have any real use for it, we can drop the parcelid feature.

In [9]:
for i in train_datasets:
    i.drop(['parcelid'], axis=1, inplace=True)

for i in test_datasets:
    i.drop(['parcelid'], axis=1, inplace=True)

## Predictions - Attempt 1 - k-NN algorithm

### Why k-NN? - Introduction 
I chose the k-NN algorithm because, usually, the house construction doesn't happen randomly. It's really unusual that a private party builds his own house, with his own money, and wherever he likes: it's more likely that the municipality's dedicated office decides where and how the houses of a given zone are buildt. 

Therefore, i think is safe to assume that houses of a given zone will have similar prices. The k-NN hopefully will help achiving this target, especially if we tune the geolocalization features with a greater weight over the other ones. 

This attempt will use the [ScikitLearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html#sklearn.neighbors.KNeighborsRegressor) of the k-NN algorithm for prediction.

The first attempt will be conducted with the parameter "weights" set as "uniform", the second one will use the value "distance".

The model will be tested with a number of neighbors beetween 6 and 8, because usually these are the value that yield the best results.

The following snippet contains the functions that wraps the described procedure.

In [10]:
n_neighbors = [6,7,8]
    
def train_test_kNN(x_train, y_train, x_test, y_test, n_neighbors, w='uniform'):
    knn = neighbors.KNeighborsRegressor(n_neighbors, weights=w)
    model = knn.fit(x_train, y_train)
    prediction = model.predict(x_test)
    scores = model.score(x_test, y_test)
    data = {'x_train': x_train,
            'y_train': y_train,
            'x_test': x_test,
            'y_test': y_test,
            'n_neighbors': n_neighbors, 
            'weights': w,
            'prediction': prediction,
            'score' : scores,
           }
    return data

In [ ]:
train = train_datasets[1]
test = test_datasets[1]
to_Y = ['logerror']
to_X = list(set(train.columns) - set('logerror'))
result = train_test_kNN(train[to_X], train[to_Y], test[to_X], test[to_Y], 5)

In [ ]:
results = []
types = ['distance', 'uniform']
to_Y = ['logerror']
to_X = list(set(train.columns) - set('logerror'))
for train in train_datasets:
    for test in test_datasets:
        for n in n_neighbors:
            for w in types:
                results.append(train_test_kNN(train[to_X], train[to_Y], test[to_X], test[to_Y], n, w))       

### How did it go? - Evaluation
After obtaining the results, we can proceed with the evaluation of the results.

In order to keep this notebook as clean as possible, the evaluation will be done with the built-in evaluator of the KNeighborsRegressor object. The built-in functions uses the $R^2$ index, the coefficient of determination that is $1 - \frac{u}{v}$, where $u$ is the residual sum of squares and $v$ is the total sum of squares.

In [1]:
def show_results_kNN(data):
    for i in data:
        parameters = i
        if parameters['weights'] == 'uniform':
            col = 'darkorange'
        else:
            col = 'blue'
        plt.scatter(parameters['n_neighbors'], parameters['score'], color=col, label='data')
    plt.title(f"KNeighborsRegressor")
    plt.show()

# plt.subplot(5, 2)

In [ ]:
show_results_kNN(results)

Basing on these results, it seems like the best results are yielded by using the weights parameter set as ‘distance’ (it weights points by the inverse of their distance, so closer neighbors of a query point will have a greater influence).

It is necessary to remind that by default the "distance" used in those cases is the Minkowski distance. With other parameters, it may have yielded other results, but for the purpose of this project is a reasonable choice.

Let's focus on the results of those experiments that yielded the best results, in order to obtain more insights.

In [ ]:
weighted_results = results.filter(lambda r: r['weights'] == 'distance')
show_results_kNN(weighted_results)

## Predictions - Attempt 2 - Linear Regression

### Why Linear Regression? - Introduction
The idea behind the adoption of the LinReg model is correlated to the low integrity of the inititial dataset. 

In contrast with the previously analyzed model, this is an attempt to see what would happen with an "assumption-free" model. It is expected that this type of analysis will underline some unseen correlations, and also will produce some interesting predictions.

This test will be conducted with the [ScikitLearn implementation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) of the LinearRegression.

In [ ]:
def train_test_LinReg(x_train, y_train, x_test, y_test):
    model = LinearRegression().fit(x_train, y_train)
    predictions = model.predict(x_test)
    data = {
        'x_train': x_train,
        'y_train': y_train,
        'x_test': x_test,
        'y_test': y_test,
        'predictions': predictions,
        'R_sq': model.score(y_test, predictions)
        'MSE': mean_squared_error(y_test, predictions)
    }
    return data


In [ ]:
res = []
to_Y = ['logerror']
to_X = list(set(train.columns) - set('logerror'))
for train in train_datasets:
    for test in test_datasets:
        res.append(train_test_LinReg(train[to_X], train[to_Y], test[to_X], test[to_Y]))

### How did it go? - Evaluation
We can now proceed with the evaluation of the results.

The following tests will be used:


*  Mean Squared Error
*  R-squared index


The evaluations will be done by using the [ScikitLearn Metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) module.

In [ ]:
def show_results_LinReg(results):
    for i in range(len(results)):
        parameters = results[i]
        plt.scatter(i, parameters['R_sq'], color="darkorange", label="data")
        plt.scatter(i, parameters['MSE'], color="darkorange", label="data")
    plt.legend()
    plt.title(f"LinearRegressor")
    plt.tight_layout()
    plt.show()


In [ ]:
show_results_LinReg(i)

### Conclusions